In [15]:
import sys
import os

# Add the parent directory (project root) to Python path
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
print(f"Added {project_root} to Python path")
print(f"Current working directory: {os.getcwd()}")
print(f"Python path includes: {[p for p in sys.path if 'zipline-engine' in p]}")

Added /home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine to Python path
Current working directory: /home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/examples
Python path includes: ['/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.zipline', '/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine', '/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.venv/lib/python3.10/site-packages']


In [16]:
# Register the NSE bundle by importing the extension
# This loads our custom bundle registration
import os
os.environ['ZIPLINE_ROOT'] = '/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.zipline'

# Import the extension which registers our bundle
import sys
zipline_root = '/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.zipline'
if zipline_root not in sys.path:
    sys.path.insert(0, zipline_root)

# This will register the nse-equities-bundle
import extension

# Verify bundle is registered using the core module
from zipline.data.bundles.core import bundles
print("Available bundles:")
for bundle_name in bundles.keys():
    print(f"  - {bundle_name}")
    
# Check specifically for our bundle
if 'nse-local-minute-bundle' in bundles:
    print("\n✅ NSE bundle is registered and ready to use!")
else:
    print("\n❌ NSE bundle not found in registered bundles")
    print("Available bundles:", list(bundles.keys()))

Available bundles:
  - quandl
  - quantopian-quandl
  - csvdir
  - nse-local-minute-bundle
  - nse-daily-bundle

✅ NSE bundle is registered and ready to use!


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from engine.zipline_runner import TradingEngine
from engine.base_strategy import TradingConfig

# Import Zipline API functions
from zipline.api import (
    symbol, 
    schedule_function, 
    date_rules, 
    time_rules,
    order_target_percent,
    order_target_value,
    record,
    get_datetime
)

In [18]:
from zipline.api import (
    order_target,
    record,
    symbol,
    schedule_function,
    date_rules,
    time_rules,
    get_datetime
)
from zipline.finance.execution import MarketOrder
import pandas as pd
import numpy as np

class ORBStrategy:
    """
    Opening Range Breakout (ORB) Strategy for NSE minute data (Zipline Reloaded):
    1. Uses 30-minute opening range candle (9:15-9:45)
    2. Takes position on 15-minute timeframe breakout
    3. Long position: if 15min close > ORB high
    4. Short position: if 15min close < ORB low
    5. Stop loss: ORB low for long, ORB high for short
    6. Target: 2R (2 times the risk)
    7. Exits all positions at 3:10 PM
    """
    
    def __init__(self):
        """Initialize strategy parameters"""
        # No config parameter needed in initialization
        self.orb_start_time = "09:15"
        self.orb_end_time = "09:45"
        self.market_close_time = "15:10"
        self.risk_reward_ratio = 2.0
        self.position_size = 0.95  # Use 95% of available capital
        
    def initialize(self, context):
        """Initialize the ORB strategy"""
        print("Initializing ORB Strategy for NSE minute data...")
        
        # Define our trading universe
        context.universe = [
            symbol('BAJFINANCE'),  # Equity
            symbol('BANKNIFTY'),   # Index
            symbol('HDFCBANK'),    # Equity
            symbol('RELIANCE'),    # Equity
            symbol('SBIN')         # Equity
        ]
        
        # Primary trading symbol
        context.primary_symbol = symbol('BAJFINANCE')
        
        # Initialize strategy variables
        self.reset_daily_variables(context)
        
        # Schedule ORB establishment at 9:45 AM
        schedule_function(
            self.establish_orb,
            date_rules.every_day(),
            time_rules.market_open(minutes=30)
        )
        
        # Schedule breakout checks every 15 minutes from 9:45 AM to 3:00 PM
        for minutes_after_open in range(45, 15*12, 15):
            schedule_function(
                self.check_breakout,
                date_rules.every_day(),
                time_rules.market_open(minutes=minutes_after_open)
            )
        
        # Schedule end-of-day close at 3:10 PM
        schedule_function(
            self.close_all_positions,
            date_rules.every_day(),
            time_rules.market_close(minutes=20)
        )
        
    def reset_daily_variables(self, context):
        """Reset all daily trading variables"""
        context.orb_high = None
        context.orb_low = None
        context.orb_established = False
        context.position_taken = False
        context.entry_price = None
        context.stop_loss = None
        context.target = None
        context.position_type = None  # 'long' or 'short'
        context.current_day = None
        
    def establish_orb(self, context, data):
        """Calculate the Opening Range Breakout levels (9:15-9:45)"""
        if not data.can_trade(context.primary_symbol):
            return
            
        try:
            # Get 30 minutes of 1m bars starting from market open
            bars = data.history(
                context.primary_symbol,
                ['high', 'low'],
                bar_count=30,
                frequency='1m',
                ffill=True
            )
            
            if len(bars) >= 30:
                context.orb_high = bars['high'].max()
                context.orb_low = bars['low'].min()
                context.orb_established = True
                
                print(f"\nORB Established at {get_datetime().time()}")
                print(f"Symbol: {context.primary_symbol.symbol}")
                print(f"ORB High: {context.orb_high:.2f}")
                print(f"ORB Low: {context.orb_low:.2f}")
                print(f"Range: {context.orb_high - context.orb_low:.2f}")
                
                record(
                    orb_high=context.orb_high,
                    orb_low=context.orb_low,
                    orb_range=context.orb_high - context.orb_low
                )
                
        except Exception as e:
            print(f"Error establishing ORB: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def check_breakout(self, context, data):
        """Check for breakout signals every 15 minutes"""
        if not context.orb_established or context.position_taken:
            return
            
        if not data.can_trade(context.primary_symbol):
            return
            
        try:
            # Get latest 15-minute closing price
            recent_close = data.history(
                context.primary_symbol,
                'close',
                bar_count=1,
                frequency='15m',
                ffill=True
            ).iloc[-1]
            
            current_time = get_datetime()
            
            # Long breakout condition
            if recent_close > context.orb_high:
                self.enter_position(context, data, recent_close, 'long')
                
            # Short breakout condition
            elif recent_close < context.orb_low:
                self.enter_position(context, data, recent_close, 'short')
                
        except Exception as e:
            print(f"Error checking breakout: {str(e)}")
    
    def enter_position(self, context, data, entry_price, position_type):
        """Enter either long or short position with proper risk management"""
        context.entry_price = entry_price
        context.position_type = position_type
        context.position_taken = True
        
        # Set risk parameters based on position type
        if position_type == 'long':
            context.stop_loss = context.orb_low
            risk_per_share = entry_price - context.stop_loss
            context.target = entry_price + (self.risk_reward_ratio * risk_per_share)
        else:  # short
            context.stop_loss = context.orb_high
            risk_per_share = context.stop_loss - entry_price
            context.target = entry_price - (self.risk_reward_ratio * risk_per_share)  # Fixed typo here
        
        # Calculate position size
        risk_capital = context.portfolio.portfolio_value * self.position_size
        shares = int(risk_capital / risk_per_share)
        
        # Execute order
        order_amount = shares if position_type == 'long' else -shares
        order_target(context.primary_symbol, order_amount)
        
        # Log trade details
        print(f"\n{'🟢 LONG' if position_type == 'long' else '🔴 SHORT'} ENTRY")
        print(f"Time: {get_datetime().time()}")
        print(f"Entry: {entry_price:.2f}")
        print(f"Stop: {context.stop_loss:.2f}")
        print(f"Target: {context.target:.2f}")
        print(f"Shares: {abs(shares)} ({'Long' if position_type == 'long' else 'Short'})")
        
        record(
            entry_price=entry_price,
            position_type=1 if position_type == 'long' else -1,
            position_size=shares
        )
    
    def handle_data(self, context, data):
        """Monitor positions every minute - this is the main handler"""
        # Check if we have a position to manage
        if context.position_taken:
            self.manage_position(context, data)
    
    def manage_position(self, context, data):
        """Monitor open positions for exits"""
        if not context.position_taken:
            return
            
        current_price = data.current(context.primary_symbol, 'price')
        
        # Check exit conditions
        if context.position_type == 'long':
            if current_price <= context.stop_loss:
                self.exit_position(context, data, current_price, "SL Hit")
            elif current_price >= context.target:
                self.exit_position(context, data, current_price, "Target Hit")
        else:  # short
            if current_price >= context.stop_loss:
                self.exit_position(context, data, current_price, "SL Hit")
            elif current_price <= context.target:
                self.exit_position(context, data, current_price, "Target Hit")
    
    def exit_position(self, context, data, exit_price, reason):
        """Exit the current position"""
        order_target(context.primary_symbol, 0)
        
        # Calculate P&L
        pnl = (exit_price - context.entry_price) if context.position_type == 'long' \
              else (context.entry_price - exit_price)
        pnl_pct = (pnl / context.entry_price) * 100
        
        print(f"\n🔶 POSITION CLOSED: {reason}")
        print(f"Exit Price: {exit_price:.2f}")
        print(f"P&L: {pnl:.2f} ({pnl_pct:.2f}%)")
        
        record(
            exit_price=exit_price,
            pnl=pnl,
            exit_reason=reason
        )
        
        # Reset position tracking
        context.position_taken = False
        context.position_type = None
    
    def close_all_positions(self, context, data):
        """Close all positions at market close"""
        if context.position_taken:
            current_price = data.current(context.primary_symbol, 'price')
            self.exit_position(context, data, current_price, "EOD Close")
    
    def before_trading_start(self, context, data):
        """Prepare for new trading day"""
        current_date = get_datetime().date()
        if context.current_day != current_date:
            self.reset_daily_variables(context)
            context.current_day = current_date
            print(f"\n📅 New Trading Day: {current_date}")

In [19]:
def run_orb_backtest():
    """Run backtest for the ORB strategy and return performance metrics"""
    # Configuration for ORB strategy
    config = TradingConfig(
        start_date="2021-01-01",
        end_date="2022-12-31", 
        capital_base=1000000,  # ₹10 lakhs
        commission_cost=0.001,
        output_dir='./results/orb'
    )
    
    # Initialize engine and ORB strategy
    engine = TradingEngine(config)
    orb_strategy = ORBStrategy()  # No config passed here
    
    # Run backtest
    print("Running ORB Strategy Backtest...")
    results = engine.run_backtest(orb_strategy)
    
    # Performance Analysis
    returns = results['returns']
    cumulative_returns = (1 + returns).cumprod()
    
    # Calculate performance metrics
    performance = {
        'total_return': cumulative_returns.iloc[-1] - 1,
        'annual_return': (1 + returns.mean()) ** 252 - 1,
        'volatility': returns.std() * np.sqrt(252),
        'sharpe_ratio': returns.mean() / returns.std() * np.sqrt(252) if returns.std() != 0 else 0,
        'max_drawdown': (cumulative_returns / cumulative_returns.expanding().max() - 1).min(),
        'win_rate': (returns > 0).sum() / len(returns) if len(returns) > 0 else 0,
        'winning_days': (returns > 0).sum(),
        'losing_days': (returns < 0).sum(),
        'total_days': len(returns),
        'best_day': returns.max(),
        'worst_day': returns.min(),
        'positive_months': (returns.resample('M').apply(lambda x: (1 + x).prod() - 1) > 0).sum(),
        'total_months': len(returns.resample('M')),
        'best_month': returns.resample('M').apply(lambda x: (1 + x).prod() - 1).max(),
        'worst_month': returns.resample('M').apply(lambda x: (1 + x).prod() - 1).min()
    }
    
    # Print performance summary
    print("\n" + "="*60)
    print("ORB STRATEGY PERFORMANCE SUMMARY")
    print("="*60)
    
    print(f"📊 RETURN METRICS:")
    print(f"   Total Return:      {performance['total_return']:.2%}")
    print(f"   Annualized Return: {performance['annual_return']:.2%}")
    print(f"   Volatility:        {performance['volatility']:.2%}")
    print(f"   Sharpe Ratio:      {performance['sharpe_ratio']:.2f}")
    print(f"   Max Drawdown:      {performance['max_drawdown']:.2%}")
    
    print(f"\n📈 TRADING METRICS:")
    print(f"   Win Rate:          {performance['win_rate']:.2%}")
    print(f"   Winning Days:      {performance['winning_days']}")
    print(f"   Losing Days:       {performance['losing_days']}")
    print(f"   Total Trading Days: {performance['total_days']}")
    print(f"   Best Day:          {performance['best_day']:.2%}")
    print(f"   Worst Day:         {performance['worst_day']:.2%}")
    
    print(f"\n📅 MONTHLY PERFORMANCE:")
    print(f"   Positive Months:   {performance['positive_months']}/{performance['total_months']} ({performance['positive_months']/performance['total_months']:.1%})")
    print(f"   Best Month:        {performance['best_month']:.2%}")
    print(f"   Worst Month:       {performance['worst_month']:.2%}")
    
    print("="*60)
    
    return {
        'results': results,
        'engine': engine,
        'performance': performance
    }

In [20]:
# Let's test the basic data availability first
from zipline.data.bundles import load
from zipline.utils.calendar_utils import get_calendar
import pandas as pd

try:
    # Load the bundle
    bundle = load('nse-local-minute-bundle')
    
    # Get the trading calendar
    trading_calendar = get_calendar('XBOM')
    
    # Get some sample assets
    asset_finder = bundle.asset_finder
    assets = asset_finder.retrieve_all(asset_finder.sids)
    
    print(f"📊 Found {len(assets)} assets in bundle:")
    for asset in assets:
        print(f"  - {asset.symbol} (sid: {asset.sid})")
        
    # Check if BAJFINANCE is available
    bajfinance_asset = None
    for asset in assets:
        if asset.symbol == 'BAJFINANCE':
            bajfinance_asset = asset
            break
    
    if bajfinance_asset:
        print(f"\n✅ BAJFINANCE found: {bajfinance_asset}")
        
        # Check data availability
        start_date = pd.Timestamp('2021-01-01', tz='UTC')
        end_date = pd.Timestamp('2021-01-10', tz='UTC')
        
        # Get daily data
        daily_reader = bundle.equity_daily_bar_reader
        daily_data = daily_reader.load_raw_arrays(
            ['open', 'high', 'low', 'close', 'volume'],
            start_date.date(),
            end_date.date(),
            [bajfinance_asset]
        )
        
        print(f"📅 Daily data shape: {daily_data[0].shape}")
        print(f"📊 Sample daily data (first 5 days):")
        for i in range(min(5, len(daily_data[0][:, 0]))):
            if daily_data[0][i, 0] > 0:  # Only show days with data
                print(f"   Day {i}: Open={daily_data[0][i, 0]:.2f}, High={daily_data[1][i, 0]:.2f}, Low={daily_data[2][i, 0]:.2f}, Close={daily_data[3][i, 0]:.2f}")
        
        print("\n✅ Data access successful! Now testing the strategy...")
        
    else:
        print("❌ BAJFINANCE not found in bundle assets")
        
except Exception as e:
    print(f"❌ Error accessing data: {e}")
    import traceback
    traceback.print_exc()

# Now let's run the backtest with debug output
def run_orb_backtest():
    """Run backtest for the ORB strategy and return performance metrics"""
    # Configuration for ORB strategy
    config = TradingConfig(
        start_date="2021-01-01",
        end_date="2021-01-31",  # Shorter period for debugging
        capital_base=1000000,  # ₹10 lakhs
        commission_cost=0.001,
        output_dir='./results/orb'
    )
    
    # Initialize engine and ORB strategy
    engine = TradingEngine(config)
    orb_strategy = ORBStrategy()  # No config passed here
    
    # Run backtest
    print("Running ORB Strategy Backtest...")
    results = engine.run_backtest(orb_strategy)
    
    # Performance Analysis
    returns = results['returns']
    cumulative_returns = (1 + returns).cumprod()
    
    # Calculate performance metrics
    performance = {
        'total_return': cumulative_returns.iloc[-1] - 1,
        'annual_return': (1 + returns.mean()) ** 252 - 1,
        'volatility': returns.std() * np.sqrt(252),
        'sharpe_ratio': returns.mean() / returns.std() * np.sqrt(252) if returns.std() != 0 else 0,
        'max_drawdown': (cumulative_returns / cumulative_returns.expanding().max() - 1).min(),
        'win_rate': (returns > 0).sum() / len(returns) if len(returns) > 0 else 0,
        'winning_days': (returns > 0).sum(),
        'losing_days': (returns < 0).sum(),
        'total_days': len(returns),
        'best_day': returns.max(),
        'worst_day': returns.min(),
        'positive_months': (returns.resample('M').apply(lambda x: (1 + x).prod() - 1) > 0).sum(),
        'total_months': len(returns.resample('M')),
        'best_month': returns.resample('M').apply(lambda x: (1 + x).prod() - 1).max(),
        'worst_month': returns.resample('M').apply(lambda x: (1 + x).prod() - 1).min()
    }
    
    # Print performance summary
    print("\n" + "="*60)
    print("ORB STRATEGY PERFORMANCE SUMMARY")
    print("="*60)
    
    print(f"📊 RETURN METRICS:")
    print(f"   Total Return:      {performance['total_return']:.2%}")
    print(f"   Annualized Return: {performance['annual_return']:.2%}")
    print(f"   Volatility:        {performance['volatility']:.2%}")
    print(f"   Sharpe Ratio:      {performance['sharpe_ratio']:.2f}")
    print(f"   Max Drawdown:      {performance['max_drawdown']:.2%}")
    
    print(f"\n📈 TRADING METRICS:")
    print(f"   Win Rate:          {performance['win_rate']:.2%}")
    print(f"   Winning Days:      {performance['winning_days']}")
    print(f"   Losing Days:       {performance['losing_days']}")
    print(f"   Total Trading Days: {performance['total_days']}")
    print(f"   Best Day:          {performance['best_day']:.2%}")
    print(f"   Worst Day:         {performance['worst_day']:.2%}")
    
    print(f"\n📅 MONTHLY PERFORMANCE:")
    print(f"   Positive Months:   {performance['positive_months']}/{performance['total_months']} ({performance['positive_months']/performance['total_months']:.1%})")
    print(f"   Best Month:        {performance['best_month']:.2%}")
    print(f"   Worst Month:       {performance['worst_month']:.2%}")
    
    # Additional debug info
    print(f"\n🔍 DEBUG INFO:")
    print(f"   Results type: {type(results)}")
    print(f"   Results keys: {list(results.keys()) if isinstance(results, dict) else 'N/A'}")
    
    # Check if we have recorded values in the main dataframe
    if 'performance' in results:
        perf_data = results['performance']
        print(f"   Performance data shape: {perf_data.shape if hasattr(perf_data, 'shape') else 'N/A'}")
        if hasattr(perf_data, 'columns'):
            print(f"   Performance columns: {list(perf_data.columns)}")
            
            # Check for recorded values
            if 'orb_high' in perf_data.columns:
                orb_records = perf_data['orb_high'].dropna()
                print(f"   ORB established on {len(orb_records)} days")
                if len(orb_records) > 0:
                    print(f"   Sample ORB highs: {orb_records.head().tolist()}")
            
            if 'entry_price' in perf_data.columns:
                entry_records = perf_data['entry_price'].dropna()
                print(f"   Positions entered: {len(entry_records)}")
                if len(entry_records) > 0:
                    print(f"   Sample entry prices: {entry_records.head().tolist()}")
    
    print("="*60)
    
    return {
        'results': results,
        'engine': engine,
        'performance': performance
    }

# Run the backtest
backtest_results = run_orb_backtest()

# Access the metrics:
print(f"\nQuick Summary:")
print(f"Annual Return: {backtest_results['performance']['annual_return']:.2%}")

# Access the raw results:
returns = backtest_results['results']['returns']

# Access the engine:
engine = backtest_results['engine']

Traceback (most recent call last):
  File "/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.venv/lib/python3.10/site-packages/zipline/data/bcolz_daily_bars.py", line 559, in _load_raw_arrays_date_to_index
    return self.sessions.get_loc(date)
  File "/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.venv/lib/python3.10/site-packages/pandas/core/indexes/datetimes.py", line 627, in get_loc
    raise KeyError(key)
KeyError: datetime.date(2021, 1, 1)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_67158/3272157071.py", line 37, in <module>
    daily_data = daily_reader.load_raw_arrays(
  File "/home/nakulbh/Desktop/Ankit/QuantMania/bactestingEngine/zipline-engine/.venv/lib/python3.10/site-packages/zipline/data/bcolz_daily_bars.py", line 537, in load_raw_arrays
    start_idx = self._load_raw_arrays_date_to_index(start_date)
  File "/home/nakulbh/Desktop/Ankit/QuantM

📊 Found 8 assets in bundle:
  - BAJFINANCE (sid: 0)
  - BANKNIFTY (sid: 1)
  - HDFC (sid: 2)
  - HDFCBANK (sid: 3)
  - HINDALCO (sid: 4)
  - NIFTY50 (sid: 5)
  - RELIANCE (sid: 6)
  - SBIN (sid: 7)

✅ BAJFINANCE found: Equity(0 [BAJFINANCE])
❌ Error accessing data: 2021-01-01
Running ORB Strategy Backtest...
Initializing ORB Strategy for NSE minute data...
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: history() got an unexpected keyword argument 'ffill'
Error establishing ORB: h